In [1]:
import requests
import pandas as pd
from datetime import datetime

# Getting all the airports from the RapidAPI

In [2]:
url = "https://flight-radar1.p.rapidapi.com/airports/list"

headers = {
	"X-RapidAPI-Key": "DON_T_FORGET_TO_PUT_YOUR_KEY_HERE",
	"X-RapidAPI-Host": "flight-radar1.p.rapidapi.com"
}

response = requests.get(url, headers=headers)


In [3]:
airports = response.json()

In [4]:
airports_df = pd.DataFrame(airports["rows"])
airports_df = airports_df[["name", "icao", "city", "country"]]


## Example - All big airports near NYC

In [5]:
airports_df.loc[airports_df.city == "New York"]

,name,icao,city,country
3312,New York Downtown Manhattan Heliport,KJRB,New York,United States
3313,New York John F. Kennedy International Airport,KJFK,New York,United States
3314,New York LaGuardia Airport,KLGA,New York,United States
3315,New York Newark Liberty International Airport,KEWR,New York,United States
3316,New York Stewart International Airport,KSWF,New York,United States


# Let's look for arrivals

We create a function which will take a **ICAO** as a parametr and return a Pandas DataFrame with all arrivals, which are available at the moment. is is the **unique** field

In [8]:
def bringMeTheArrivals(icao):
    # bring up the initials
    purified_arrivals = []
    direct_url = "http://api.flightradar24.com/common/v1/airport.json"
    ppp = {"code":icao, "page":1}
    
    # without headers you will get acquainted with 451 error
    headers={"User-Agent":"PostmanRuntime/7.35.0"}
    
    # getting first response to analyse the amount of work
    first_response = requests.get(direct_url, params=ppp, headers=headers)
    if first_response.status_code != 200:
        print("Something went wrong")
        return -1
    dir_json = first_response.json()
    
    # getting the amount of all pages
    pages = dir_json["result"]["response"]["airport"]['pluginData']['schedule']["arrivals"]["page"]["total"]
    page = 1
    
    # now bringing all to the cycle - we will fetch first page, look for the others, repeat until we will
    # reach the end
    while True:
        arrivals = dir_json["result"]["response"]["airport"]["pluginData"]["schedule"]["arrivals"]["data"]
        for arrival in arrivals:
            temp = arrival["flight"]
            dt_object = datetime.utcfromtimestamp(temp["time"]["scheduled"]["arrival"])
            if "Scheduled" in temp["status"]["text"]:
                temp_res ={
                    "id":temp["identification"]["row"],
                    "identification":temp["identification"]["number"]["default"],
                    "arrivalAirport":icao,
                    "departureAirport":temp["airport"]["origin"]["code"]["icao"],
                    "landing_time": dt_object.strftime('%Y-%m-%d %H:%M:%S'),
                     }
                purified_arrivals.append(temp_res)
            
        # checking if it is the last page or not
        if page == pages:
            break
        page += 1
        ppp = {"code":icao, "page":page}
        response = requests.get(direct_url, params=ppp, headers=headers)
        dir_json = response.json()
    
    # creating DataFrame, making datetime and returning
    arrivals_df = pd.DataFrame(purified_arrivals)
    arrivals_df.landing_time = pd.to_datetime(arrivals_df["landing_time"])
    return arrivals_df

### Example - arrivals at New York Newark Liberty International Airport

In [9]:
new_ark_arrivals = bringMeTheArrivals("KLGA")

In [10]:
new_ark_arrivals

,id,identification,arrivalAirport,departureAirport,landing_time
0,5441257153,AC8986,KLGA,CYYZ,2023-11-29 16:46:00
1,5444278807,ASP668,KLGA,KBWI,2023-11-29 17:20:00
2,5444544184,TFF976,KLGA,KFRG,2023-11-29 17:24:00
3,5441067002,DL2076,KLGA,KCHS,2023-11-29 17:27:00
4,5441073520,DL5059,KLGA,KBUF,2023-11-29 17:48:00
...,...,...,...,...,...
879,5441721246,UA6056,KLGA,KIAD,2023-12-01 04:23:00
880,5441665005,DL5615,KLGA,KBOS,2023-12-01 04:25:00
881,5441726459,WN43,KLGA,KHOU,2023-12-01 04:25:00
882,5441726752,WN785,KLGA,KBNA,2023-12-01 04:25:00
